In [1]:
# coding=utf-8
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from keras.utils import np_utils
from keras.models import Sequential,load_model,save_model
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras import backend as K
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score,accuracy_score
from scipy import sparse
import gc
from time import strftime, localtime

Using TensorFlow backend.


In [3]:
# 打印当前时间
def printTime():
    print(strftime("%Y-%m-%d %H:%M:%S", localtime()))
    return

In [4]:
printTime()

2019-07-19 18:08:25


In [5]:
# 读取TXT格式存储的训练集
train = np.loadtxt('../processed/stacking/train_data.txt', dtype=float, delimiter=',')
train.shape

(2010000, 157)

In [6]:
age_train = pd.read_csv(r'../data/age_train.csv',header=None)
label = age_train[1].values
print(label.shape)

(2010000,)


In [7]:
import time

seed = 7
np.random.seed(seed)

In [8]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

In [9]:
model_filePath = r'../processed/stacking/nn_model_'
currK = 0
val_index_list, score = [], []
val_probability = np.zeros((2010000,7))

In [10]:
for train_index, val_index in kfold.split(train,label):
    K.clear_session()
    trainData, trainLabel, valData, valLabel = train[train_index,:], label[train_index], train[val_index,:] , label[val_index] 
    trainLabel,valLabel = np_utils.to_categorical(trainLabel,num_classes=7),np_utils.to_categorical(valLabel,num_classes=7)
    
    print('----------------------------------------------------------------------------------------------------------------------------------')
    print(currK,'split Done!\n')

    # 全连接模型
    model = Sequential()
    model.add(Dense(80, activation='tanh', input_shape=(train.shape[1],)))
    model.add(Dense(40, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    #损失函数使用交叉熵
    adam = Adam(lr=0.0001)
    model.compile(loss='categorical_crossentropy',
                  optimizer = adam,
                  metrics=['accuracy'])
    #模型训练
    batch_size = 10240
    epochs = 1000
    early_stopping = EarlyStopping(monitor='val_loss', patience=12, verbose=1)
    bestModel = ModelCheckpoint(model_filePath + str(currK) + r'.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    hist = model.fit(trainData, trainLabel,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1,
                      shuffle=True,
                      validation_data=(valData,valLabel),
                      callbacks=[early_stopping,bestModel],
                      ) 

    print('\n',currK,'train Done!')
    printTime()
    
    K.clear_session()
    model = load_model(model_filePath + str(currK) + r'.h5')
    probability = model.predict(valData,batch_size=1024)
    val_probability[val_index,:] = probability
    
    score.append(np.max(hist.history['val_acc']))
    y_label = label[val_index]
    val_label = np.argmax(probability,axis=1) 
    print(currK,'val_acc:',accuracy_score(val_label,y_label),'\n\n')
    
    currK += 1
    K.clear_session()
    del trainData, valData, trainLabel,valLabel,model
    print('----------------------------------------------------------------------------------------------------------------------------------')
print('mean val_acc:', np.mean(score))
printTime()

----------------------------------------------------------------------------------------------------------------------------------
0 split Done!

Train on 1809000 samples, validate on 201000 samples
Epoch 1/1000
1809000/1809000 [==============================] - 7s 4us/step - loss: 1.5222 - acc: 0.5193 - val_loss: 1.1365 - val_acc: 0.6492
Epoch 2/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.9845 - acc: 0.6508 - val_loss: 0.8946 - val_acc: 0.6531
Epoch 3/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8731 - acc: 0.6529 - val_loss: 0.8593 - val_acc: 0.6548
Epoch 4/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8552 - acc: 0.6547 - val_loss: 0.8500 - val_acc: 0.6561
Epoch 5/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8480 - acc: 0.6556 - val_loss: 0.8442 - val_acc: 0.6567
Epoch 6/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8430 

Epoch 57/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8114 - acc: 0.6604 - val_loss: 0.8110 - val_acc: 0.6609
Epoch 58/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8113 - acc: 0.6604 - val_loss: 0.8109 - val_acc: 0.6610
Epoch 59/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8112 - acc: 0.6604 - val_loss: 0.8109 - val_acc: 0.6610
Epoch 60/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8111 - acc: 0.6605 - val_loss: 0.8107 - val_acc: 0.6610
Epoch 61/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8110 - acc: 0.6605 - val_loss: 0.8107 - val_acc: 0.6611
Epoch 62/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8109 - acc: 0.6603 - val_loss: 0.8106 - val_acc: 0.6609
Epoch 63/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8109 - acc: 0.6604 - val_loss: 0.8105 - val_acc: 0.6611

1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8087 - acc: 0.6607 - val_loss: 0.8089 - val_acc: 0.6611
Epoch 115/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8087 - acc: 0.6608 - val_loss: 0.8088 - val_acc: 0.6614
Epoch 116/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8086 - acc: 0.6607 - val_loss: 0.8087 - val_acc: 0.6614
Epoch 117/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8086 - acc: 0.6608 - val_loss: 0.8088 - val_acc: 0.6613
Epoch 118/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8086 - acc: 0.6608 - val_loss: 0.8088 - val_acc: 0.6614
Epoch 119/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8086 - acc: 0.6607 - val_loss: 0.8087 - val_acc: 0.6614
Epoch 120/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8086 - acc: 0.6607 - val_loss: 0.8090 - val_acc: 0.6613
Epoch 1

Epoch 171/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8076 - acc: 0.6610 - val_loss: 0.8082 - val_acc: 0.6614
Epoch 172/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8076 - acc: 0.6608 - val_loss: 0.8082 - val_acc: 0.6616
Epoch 173/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8076 - acc: 0.6609 - val_loss: 0.8080 - val_acc: 0.6617
Epoch 174/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8076 - acc: 0.6609 - val_loss: 0.8081 - val_acc: 0.6617
Epoch 175/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8075 - acc: 0.6608 - val_loss: 0.8080 - val_acc: 0.6617
Epoch 176/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8075 - acc: 0.6610 - val_loss: 0.8081 - val_acc: 0.6613
Epoch 177/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8075 - acc: 0.6609 - val_loss: 0.8081 - val_acc:

Epoch 228/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8069 - acc: 0.6611 - val_loss: 0.8078 - val_acc: 0.6617
Epoch 229/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8068 - acc: 0.6610 - val_loss: 0.8077 - val_acc: 0.6616
Epoch 230/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8069 - acc: 0.6611 - val_loss: 0.8077 - val_acc: 0.6616
Epoch 231/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8069 - acc: 0.6611 - val_loss: 0.8077 - val_acc: 0.6616
Epoch 00231: early stopping

 0 train Done!
2019-07-19 18:26:56
0 val_acc: 0.6618059701492537 


----------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------------------
1 split Done!

Train on 1809000 samples, validate on 201000 sampl

...

1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8187 - acc: 0.6595 - val_loss: 0.8151 - val_acc: 0.6601
Epoch 27/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8182 - acc: 0.6596 - val_loss: 0.8145 - val_acc: 0.6601
Epoch 28/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8178 - acc: 0.6597 - val_loss: 0.8141 - val_acc: 0.6603
Epoch 29/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8174 - acc: 0.6598 - val_loss: 0.8136 - val_acc: 0.6600
Epoch 30/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8170 - acc: 0.6598 - val_loss: 0.8133 - val_acc: 0.6603
Epoch 31/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8166 - acc: 0.6598 - val_loss: 0.8131 - val_acc: 0.6601
Epoch 32/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8163 - acc: 0.6598 - val_loss: 0.8126 - val_acc: 0.6600
Epoch 33/1000

1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8101 - acc: 0.6606 - val_loss: 0.8073 - val_acc: 0.6608
Epoch 84/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8101 - acc: 0.6606 - val_loss: 0.8072 - val_acc: 0.6610
Epoch 85/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8100 - acc: 0.6606 - val_loss: 0.8072 - val_acc: 0.6608
Epoch 86/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8100 - acc: 0.6605 - val_loss: 0.8071 - val_acc: 0.6608
Epoch 87/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8099 - acc: 0.6606 - val_loss: 0.8071 - val_acc: 0.6608
Epoch 88/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8099 - acc: 0.6606 - val_loss: 0.8070 - val_acc: 0.6606
Epoch 89/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8098 - acc: 0.6606 - val_loss: 0.8072 - val_acc: 0.6607
Epoch 90/1000

1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8086 - acc: 0.6608 - val_loss: 0.8062 - val_acc: 0.6609
Epoch 141/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8085 - acc: 0.6609 - val_loss: 0.8062 - val_acc: 0.6610
Epoch 142/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8085 - acc: 0.6608 - val_loss: 0.8062 - val_acc: 0.6607
Epoch 143/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8085 - acc: 0.6608 - val_loss: 0.8062 - val_acc: 0.6608
Epoch 144/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8085 - acc: 0.6608 - val_loss: 0.8062 - val_acc: 0.6610
Epoch 145/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8084 - acc: 0.6609 - val_loss: 0.8063 - val_acc: 0.6611
Epoch 146/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8084 - acc: 0.6608 - val_loss: 0.8061 - val_acc: 0.6609
Epoch 1

Epoch 197/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8077 - acc: 0.6609 - val_loss: 0.8058 - val_acc: 0.6607
Epoch 198/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8077 - acc: 0.6610 - val_loss: 0.8058 - val_acc: 0.6606
Epoch 199/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8077 - acc: 0.6610 - val_loss: 0.8057 - val_acc: 0.6609
Epoch 200/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8077 - acc: 0.6609 - val_loss: 0.8058 - val_acc: 0.6610
Epoch 201/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8077 - acc: 0.6611 - val_loss: 0.8056 - val_acc: 0.6610
Epoch 202/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8076 - acc: 0.6610 - val_loss: 0.8056 - val_acc: 0.6608
Epoch 203/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8077 - acc: 0.6610 - val_loss: 0.8057 - val_acc:

Epoch 254/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8071 - acc: 0.6610 - val_loss: 0.8054 - val_acc: 0.6609
Epoch 255/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8071 - acc: 0.6611 - val_loss: 0.8055 - val_acc: 0.6608
Epoch 256/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8071 - acc: 0.6611 - val_loss: 0.8055 - val_acc: 0.6608
Epoch 257/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8071 - acc: 0.6612 - val_loss: 0.8055 - val_acc: 0.6607
Epoch 258/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8071 - acc: 0.6611 - val_loss: 0.8056 - val_acc: 0.6607
Epoch 259/1000
1809000/1809000 [==============================] - 4s 2us/step - loss: 0.8071 - acc: 0.6611 - val_loss: 0.8055 - val_acc: 0.6610
Epoch 260/1000
1809000/1809000 [==============================] - 3s 2us/step - loss: 0.8070 - acc: 0.6610 - val_loss: 0.8055 - val_acc:

In [22]:
del train
gc.collect()

46880

# 测试集

In [11]:
import os

In [12]:
model_file = r'../processed/stacking/'
model_name_list = ['nn_model_0.h5',
                     'nn_model_1.h5',
                     'nn_model_2.h5',
                     'nn_model_3.h5',
                     'nn_model_4.h5',
                     'nn_model_5.h5',
                     'nn_model_6.h5',
                     'nn_model_7.h5',
                     'nn_model_8.h5',
                     'nn_model_9.h5']
model_name_list

['nn_model_0.h5',
 'nn_model_1.h5',
 'nn_model_2.h5',
 'nn_model_3.h5',
 'nn_model_4.h5',
 'nn_model_5.h5',
 'nn_model_6.h5',
 'nn_model_7.h5',
 'nn_model_8.h5',
 'nn_model_9.h5']

In [13]:
test = np.loadtxt('../processed/stacking/test_data.txt', dtype=float, delimiter=',')
test.shape

(502500, 157)

In [14]:
# 十个模型融合
proflag = True
model_Num = 0
for model_name in model_name_list:
    model = load_model(model_file + model_name)
    if proflag==True:
        probability = model.predict(test,batch_size=1024,verbose=1)
        proflag = False
    else:
        probability += model.predict(test,batch_size=1024,verbose=1)
    model_Num += 1
    print(model_Num)
    K.clear_session()
    del model

502500/502500 [==============================] - 1s 2us/step
1
502500/502500 [==============================] - 1s 2us/step
2
502500/502500 [==============================] - 1s 2us/step
3
502500/502500 [==============================] - 1s 2us/step
4
502500/502500 [==============================] - 1s 2us/step
5
502500/502500 [==============================] - 1s 2us/step
6
502500/502500 [==============================] - 1s 1us/step
7
502500/502500 [==============================] - 1s 1us/step
8
502500/502500 [==============================] - 1s 2us/step
9
502500/502500 [==============================] - 1s 1us/step
10


In [15]:
model_Num

10

In [16]:
probability /= model_Num
age = np.argmax(probability,axis=1)

In [17]:
age_test = pd.read_csv(r'../data/age_test.csv',header=None,usecols=[0])
age_test = age_test.values
type(age_test)

numpy.ndarray

In [18]:
probability.shape

(502500, 7)

In [19]:
result = np.column_stack((age_test,age))
result = pd.DataFrame(result,columns=['id','label'])
print(result.shape)
result.to_csv('../data/submission.csv',index=False,header=True)

(502500, 2)


In [20]:
# 十个模型融合
result['label'].value_counts()

3    163272
4    128392
2     97798
5     68923
6     30226
1     13889
Name: label, dtype: int64